In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('HousePrice.csv')
df_test = pd.read_csv('HousePriceTest.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Q4: Perform the preprocessing if required, scale the train and test data using standard scaler

In [4]:
cols_with_missing_vals = df.columns[df.isnull().any()].tolist()
df.drop(cols_with_missing_vals, axis = 1, inplace=True)
df_test.drop(cols_with_missing_vals, axis = 1, inplace=True)

In [5]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

objects = df.select_dtypes(['object']).columns.tolist()
encoder = OneHotEncoder()

encoder_df = pd.DataFrame(encoder.fit_transform(df[objects]).toarray())
df = df.join(encoder_df)

encoder_df = pd.DataFrame(encoder.transform(df[objects]).toarray())
df_test = df_test.join(encoder_df)

In [6]:
df.drop(objects, axis = 1, inplace=True)
df_test.drop(objects, axis = 1, inplace=True)

In [7]:
X_train, y_train = df.drop(['SalePrice'], axis=1).to_numpy(), df[['SalePrice']].to_numpy()
X_test = df_test.to_numpy()

### Q5: Split the dataset into train size of 70% and test size of 30% and Apply the Ridge and Lasso regression and fit the model containing all independent variables.

In [8]:
scaler = StandardScaler()
scaler.fit_transform(np.asarray(X_train))
scaler.transform(np.asarray(X_test))

array([[ 1.73323755, -0.87256276,  0.11076257, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 1.73561022, -0.87256276,  0.37584985, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 1.7379829 ,  0.07337496,  0.33205282, ..., -0.11785113,
         0.4676514 , -0.30599503],
       ...,
       [ 5.18784929, -0.87256276,  0.95042275, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 5.19022196,  0.66458604, -0.00759964, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 5.19259463,  0.07337496, -0.08918038, ..., -0.11785113,
         0.4676514 , -0.30599503]])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.3, random_state = 10)

In [10]:
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train, y_train)
ridge.score(X_test, y_test)

0.8684107491066856

In [11]:
lasso = linear_model.Lasso(alpha=0.5)
lasso.fit(X_train, y_train)
lasso.score(X_test, y_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 226515485744.03928, tolerance: 663740509.674582
  model = cd_fast.enet_coordinate_descent(


0.8649828216641964

### Q6: Make predictions on test data “HousePriceTest.csv” and tabulate performance of both models on unseen data.

In [12]:
lasso_predict = lasso.predict(X_test).reshape(1,-1).tolist()[0]
ridge_predict = ridge.predict(X_test).reshape(1,-1).tolist()[0]
pd.DataFrame({'lasso':lasso_predict, 'ridge':ridge_predict, 'Actual':list(y_test)})

,lasso,ridge,Actual
0,195790.825531,190370.004957,[170000]
1,221036.853392,227739.279214,[187750]
2,123014.997485,125866.256146,[137000]
3,154395.679466,153762.475764,[164500]
4,296943.474394,304846.651710,[309000]
...,...,...,...
433,187795.284600,181881.413633,[149000]
434,155790.189399,142111.085051,[145000]
435,119476.457655,123405.994102,[131500]
436,340238.270279,333653.985341,[285000]


For the HousePrice dataset, the lasso and ridge regression have the same R2 score and hence neither is better than the other in this case. 